In [1]:
import os
import shutil
import gnupg
# gnupg.__version__
from pprint import pprint
from pyspark.sql import *
%load_ext pycodestyle_magic
from pyspark import SparkContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [2]:
# %%pycodestyle
root_dir = root_dir = os.getcwd()
#os.path.dirname(os.path.realpath("./"))
program_subdir = "" #"python-data-bricks-sample"
full_program_dir = root_dir #os.path.join(root_dir, program_subdir)
print(full_program_dir)
gpg = gnupg.GPG(gnupghome=full_program_dir)
public_keys = gpg.list_keys()
private_keys = gpg.list_keys(True)
print('public keys:')
pprint(public_keys)
print('private keys:')
pprint(private_keys)

# key_data = open(os.path.join(full_program_dir, 'slim.shady.pub.asc')).read()
key_data = open('slim.shady.pub.asc').read()
# print(key_data)
import_result = gpg.import_keys(key_data)
pprint(import_result.results)
# key_data = open(os.path.join(full_program_dir, 'slim.shady.sec.asc')).read()
key_data = open('slim.shady.sec.asc').read()
import_result = gpg.import_keys(key_data)
pprint(import_result.results)

/home/david/deeplearning_cpu/python-data-bricks-sample
public keys:
[{'algo': '1',
  'cap': 'scESC',
  'compliance': '',
  'curve': '',
  'date': '1519158845',
  'dummy': '',
  'expires': '',
  'fingerprint': 'F48475940AC469A6604A15127D4CE3C056D601F5',
  'flag': '',
  'hash': '',
  'issuer': '',
  'keyid': '7D4CE3C056D601F5',
  'length': '2048',
  'origin': 'unavailable',
  'ownertrust': '-',
  'sig': '',
  'sigs': [],
  'subkey_info': {'857D0D1AA09F19F8': {'algo': '1',
                                       'cap': 'e',
                                       'compliance': '',
                                       'curve': '',
                                       'date': '1519158845',
                                       'dummy': '',
                                       'expires': '',
                                       'flag': '',
                                       'hash': '',
                                       'issuer': '',
                                       'key

In [3]:
# %%pycodestyle
save_csv = os.path.join(full_program_dir, 'out.csv')
print(save_csv)
with open(os.path.join(full_program_dir, 'titanic.csv.gpg'), 'rb') as f:
    status = gpg.decrypt_file(f, output=save_csv)
status.ok

/home/david/deeplearning_cpu/python-data-bricks-sample/out.csv


True

In [4]:
# %%pycodestyle
df = sqlContext.read.load('file://' + save_csv,
                          format='com.databricks.spark.csv',
                          header='true',
                          inferSchema='true')

In [5]:
# %%pycodestyle
df.summary()

DataFrame[summary: string, RecId: string, Name: string, PassengerClass: string, Age: string, Sex: string, Survived: string, SexCode: string]

In [6]:
# %%pycodestyle
titanic_parquet = os.path.join(full_program_dir, "titanic_parquet")
if os.path.exists(titanic_parquet):
    shutil.rmtree(titanic_parquet)
# write to parquet
df.write.parquet(titanic_parquet)

In [7]:
# %%pycodestyle
pardata = sqlContext.read.parquet(titanic_parquet)
pardata.registerTempTable("titanic")

In [8]:
# %%pycodestyle
avgage = sqlContext.sql("SELECT Avg(CAST(Age AS Double)), " +
                        " Count(*) FROM titanic")
avgcount = sqlContext.sql("SELECT CAST(Count(Age) AS Double) " +
                          "FROM titanic WHERE Age NOT IN ('NA')")

In [9]:
# %%pycodestyle
getvals = int(0.75 * avgcount.first()[0])
getvals

567

In [10]:
# %%pycodestyle
top75pctavg = sqlContext.sql("SELECT Avg(Age), Count(*) FROM (" +
                             " SELECT CAST(Age AS DOUBLE) " +
                             " From titanic WHERE Age NOT IN (\'NA\') " +
                             " ORDER BY Age DESC LIMIT " + str(getvals) + ")")

In [11]:
# %%pycodestyle
print("Average age is " + str(avgage.first()[0]))

Average age is 30.397989417989415


In [12]:
# %%pycodestyle
print("Top 75 percent Average age is " + str(top75pctavg.first()[0]))

Top 75 percent Average age is 35.85008818342152
